# 14. Experimenting with label smoothing and CEL to deal with noisy ground truth
We tried to train a network with label smoothing, which is generally done when the ground truth is noisy or involves a lot of subjectivity. The practice of label smoothing is tried for classification problem but never for a segmentation problem. We tried it for segmentation problem. It didn't seem to work well and hence training was stopped mid-way as the dice-scores were no where close to acceptable levels. As a future work, we would like to engineer a  way to segment images based on noisy ground truth

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
from skimage import io, transform
import matplotlib.image as mpimg
from PIL import Image
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import scipy
import random
import pickle
import scipy.io as sio
import itertools
from scipy.ndimage.interpolation import shift
import copy
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.ion()

# Import Dataloader Class and other utilities

In [2]:
from dataloader_2d import *
from dataloader_3d import *

# Build Data loader objects

In [3]:
train_path = '/beegfs/ark576/new_knee_data/train'
val_path = '/beegfs/ark576/new_knee_data/val'
test_path = '/beegfs/ark576/new_knee_data/test'

train_file_names = sorted(pickle.load(open(train_path + '/train_file_names.p','rb')))
val_file_names = sorted(pickle.load(open(val_path + '/val_file_names.p','rb')))
test_file_names = sorted(pickle.load(open(test_path + '/test_file_names.p','rb')))

transformed_dataset = {'train': KneeMRIDataset(train_path,train_file_names, train_data= True, flipping=False, normalize= True),
                       'validate': KneeMRIDataset(val_path,val_file_names, normalize= True),
                       'test': KneeMRIDataset(test_path,test_file_names, normalize= True)
                                          }

dataloader = {x: DataLoader(transformed_dataset[x], batch_size=5,
                        shuffle=True, num_workers=0) for x in ['train', 'validate','test']}
data_sizes ={x: len(transformed_dataset[x]) for x in ['train', 'validate','test']}

In [4]:
im, seg_F, seg_P, seg_T,_ = next(iter(dataloader['train']))

# Find Max and min values of Images (all 7 contrasts), of Fractional Anisotropy maps and of Mean Diffusivity maps for image normalization

In [5]:
min_fa = np.inf
min_md = np.inf
min_image = np.inf
max_fa = 0
max_md = 0
max_image = 0
for data in dataloader['train']:
    if min_fa > torch.min(data[0][:,7,:,:]):
        min_fa = torch.min(data[0][:,7,:,:])
    if min_md > torch.min(data[0][:,8,:,:]):
        min_md = torch.min(data[0][:,8:,:])
    if min_image > torch.min(data[0][:,:7,:,:]):
        min_image = torch.min(data[0][:,:7,:,:])
    if max_fa < torch.max(data[0][:,7,:,:]):
        max_fa = torch.max(data[0][:,7,:,:])
    if max_md < torch.max(data[0][:,8,:,:]):
        max_md = torch.max(data[0][:,8,:,:])
    if max_image < torch.max(data[0][:,:7,:,:]):
        max_image = torch.max(data[0][:,:7,:,:])
norm_values = (max_image, min_image, max_fa, min_fa, max_md, min_md)

# Import Models

In [6]:
from unet_3d import *
from unet_basic_dilated import *
from vnet import *
from ensemble_model import *

In [7]:
seg_sum = torch.zeros(3)
for i, data in enumerate(dataloader['train']):
    input, segF, segP, segT,_ = data
    seg_sum[0] += torch.sum(segF)
    seg_sum[1] += torch.sum(segP)
    seg_sum[2] += torch.sum(segT)
mean_s_sum = seg_sum/i

## Import Loss functions and all other utility functions like functions for saving models, for visualizing images, etc.

In [8]:
from utils import *

## Import all the Training and evaluate functions to evaluate the models

In [10]:
from train_2d import *
from train_3d import *
from train_ensemble import *
from evaluate_2d import *
from evaluate_3d import *
from evaluate_ensemble import *

# 14. Experimenting with label smoothing and CEL to deal with noisy ground truth
We tried to train a network with label smoothing, which is generally done when the ground truth is noisy or involves a lot of subjectivity. The practice of label smoothing is tried for classification problem but never for a segmentation problem. We tried it for segmentation problem. It didn't seem to work well and hence training was stopped mid-way as the dice-scores were no where close to acceptable levels. As a future work, we would like to engineer a  way to segment images based on noisy ground truth

In [68]:
unet_exp_noisy = Unet_dilated_small(9,4,int_var=40,dilated=False).cuda()

In [69]:
count_parameters(unet_exp_noisy)

410524

In [70]:
optimizer_unet_exp_noisy = optim.Adam(unet_exp_noisy.parameters(),lr = 1e-4)

In [80]:
unet_exp_noisy, loss_hist_unet_exp, dc_hist_0_unet_exp, \
dc_hist_1_unet_exp, dc_hist_2_unet_exp = train_model(unet_exp_noisy, optimizer_unet_exp_noisy,
                                                     dataloader,data_sizes,5,'new_data_unet_exp_noisy_1',
                                                     num_epochs= 50, verbose = True, 
                                                     dice_loss = dice_loss_3,noisy_labels = True)

Epoch: 0, Phase: train, epoch loss: 0.0069, Dice Score (class 0): 0.4548, Dice Score (class 1): 0.3627,Dice Score (class 2): 0.2732
----------
Epoch: 0, Phase: validate, epoch loss: 0.0107, Dice Score (class 0): 0.4652, Dice Score (class 1): 0.2242,Dice Score (class 2): 0.2691
----------
Epoch: 1, Phase: train, epoch loss: 0.0066, Dice Score (class 0): 0.4464, Dice Score (class 1): 0.3631,Dice Score (class 2): 0.2697
----------
Epoch: 1, Phase: validate, epoch loss: 0.0116, Dice Score (class 0): 0.4738, Dice Score (class 1): 0.1400,Dice Score (class 2): 0.2687
----------
Epoch: 2, Phase: train, epoch loss: 0.0063, Dice Score (class 0): 0.4417, Dice Score (class 1): 0.3004,Dice Score (class 2): 0.2691
----------
Epoch: 2, Phase: validate, epoch loss: 0.0083, Dice Score (class 0): 0.4532, Dice Score (class 1): 0.1267,Dice Score (class 2): 0.2386
----------
Epoch: 3, Phase: train, epoch loss: 0.0058, Dice Score (class 0): 0.4365, Dice Score (class 1): 0.3346,Dice Score (class 2): 0.2603
-

Epoch: 28, Phase: validate, epoch loss: 0.0089, Dice Score (class 0): 0.3152, Dice Score (class 1): 0.1829,Dice Score (class 2): 0.2807
----------
Epoch: 29, Phase: train, epoch loss: 0.0028, Dice Score (class 0): 0.3628, Dice Score (class 1): 0.2731,Dice Score (class 2): 0.2147
----------
Epoch: 29, Phase: validate, epoch loss: 0.0089, Dice Score (class 0): 0.4576, Dice Score (class 1): 0.1791,Dice Score (class 2): 0.2617
----------
Epoch: 30, Phase: train, epoch loss: 0.0025, Dice Score (class 0): 0.3856, Dice Score (class 1): 0.3332,Dice Score (class 2): 0.2106
----------
Epoch: 30, Phase: validate, epoch loss: 0.0088, Dice Score (class 0): 0.4605, Dice Score (class 1): 0.2360,Dice Score (class 2): 0.2981
----------
Epoch: 31, Phase: train, epoch loss: 0.0022, Dice Score (class 0): 0.4015, Dice Score (class 1): 0.3838,Dice Score (class 2): 0.2237
----------
Epoch: 31, Phase: validate, epoch loss: 0.0103, Dice Score (class 0): 0.4688, Dice Score (class 1): 0.4179,Dice Score (class 2)

KeyboardInterrupt: 

In [ ]:
# torch.save(model_gen_dilated_l4_n2_new_data_dp,'new_data_dilated_net_l4_n2_nd_dp_1')
pickle.dump(loss_hist_unet_exp, open('loss_hist_new_data_unet_exp_noisy_1','wb'))
pickle.dump(dc_hist_0_unet_exp, open('dc_hist_0_new_data_unet_exp_noisy_1','wb'))
pickle.dump(dc_hist_1_unet_exp, open('dc_hist_1_new_data_unet_exp_noisy_1','wb'))
pickle.dump(dc_hist_2_unet_exp, open('dc_hist_2_new_data_unet_exp_noisy_1','wb'))

In [ ]:
plot_hist(loss_hist_unet_exp,'Loss')

In [ ]:
evaluate(unet_exp_noisy, dataloader, data_sizes, 5, 'validate', dice_loss=dice_loss_3, noisy_labels = True)